#### Importing libs

In [1]:
import pandas as pd
import numpy as np

# Data Import

##### Importing features

In [2]:
sheet_id = "1xOPRcRkJrGzxb65dieEHrWHnqXYrrSpx7Ack3pr6Pdw"
features = pd.read_excel(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx", sheet_name = None)
print(features.keys()) # Empresas

dict_keys(['BOMBRIL', 'DOMMO', 'MRV', 'HOTEIS OTHON', 'JOÃO FORTES', 'MANGELS', 'ENERGISA', '3R PETROLEUM', 'ENEVA', 'ETERNIT', 'EUCATEX', 'FER HERINGER', 'GPC PART', 'LUPATECH', 'MMX MINER', 'OI', 'OSX BRASIL', 'PDG REALT', 'PET MANGUINHOS', 'POMIFRUTAS', 'RECRUSUL', 'REDE ENERGIA', 'SANSUY', 'SARAIVA', 'TECNOSOLO', 'TEKA', 'VIVER', 'WETZEL', 'BRASKEM', 'CEMIG', 'DURATEX', 'EZTEC', 'INDS ROMI', 'LITEL', 'MARCOPOLO', 'METAL IGUAÇU', 'SLC AGRICOLA', 'SONDOTECNICA', 'ULTRAPAR'])


##### Importing targets

In [15]:
sheet_id = "116cM2eSTve3UHHYESOdWXRiYxx8gGAPPLlonNDNIn6M"
target = pd.read_excel(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")

# Alterando índices
target.index = target["Empresa"]
target.drop(["Empresa"], axis = 1, inplace = True)

# Montando o alvo
print("Classificações: ",list(set(target["Classificação"])))
target["Alvo"] = np.where(target["Classificação"] == "Não Falida", 0, 1)

target.head()

Classificações:  ['Falida', 'Não Falida', 'Recuperação Judicial']


,Ticker,Setor,Classificação,Alvo
Empresa,,,,
BOMBRIL,BOBR,Consumo não Cíclico,Recuperação Judicial,1
DOMMO,DMMO,"Petróleo, Gás e Biocombustíveis",Recuperação Judicial,1
ENEVA,ENEV,Utilidade Pública,Recuperação Judicial,1
ETERNIT,ETER,Bens Industriais,Recuperação Judicial,1
EUCATEX,EUCA,Materiais Básicos,Recuperação Judicial,1


##### Validation

In [4]:
res = 0
miss = []
for i in list(features.keys()):
    if i in list(target["Empresa"].values):
        res += 1
    else:
        miss.append(i)

print("Número de Empresas (Target):", res)
print("Número de Empresas (Features):", len(features.keys())) # Numero de empresas
print("Empresas Faltando:", miss)

Número de Empresas (Target): 39
Número de Empresas (Features): 39
Empresas Faltando: []


# Data Wrangling

Próximos passos: 
* tratar NA's

In [19]:
# Corte Transversal ou Cross-Sectional (analisa dados em um momento específico) -> oposto do longitudinal (série temporal)
indicadores = list(features["BOMBRIL"].iloc[35:, 0])

def wrangle(ano_pre_fal):
    # monta df para 1 a 5 anos pré-falência
    
    df = pd.DataFrame(index = features.keys())

    for indicador in indicadores:
        coluna = []
        for empresa in features.keys():
            data = features[empresa]
            inds = data.columns[0]
            val_ind = data[data[inds] == indicador].iloc[0, 6-ano_pre_fal]
            coluna.append(val_ind)

        df[indicador] = coluna
    
    df = df.join(target["Alvo"]) # Acrescenta os alvos
    
    return df

In [20]:
df_5y = wrangle(5)
df_5y.head()

,Dispon/PC,Dispon/Receita Liq Operacional,Dispon/AC,Dispon/AT,AC/PC,AC/AT,PC/AT,PC/Receita Liq Operacional,Estoque/AT,Quick assets/AT,...,VA/IMOBILIZADO,VA/AT,VA/RLO,Contas a pagar/RLO,AC/RLO,Estoque/RLO,NOWC/RLO,Contas a receber/RLO,RLO/AT,Alvo
BOMBRIL,0.016553,0.0146029,0.0415163,0.0035729,0.398709,0.0860601,0.215847,0.882195,0.00776411,0.078296,...,0.8985,0.0549602,0.22463,0.106932,0.35174,0.031733,0.131856,0.291518,0.24467,1
DOMMO,738.67,#DIV/0!,0.973268,0.387517,758.958,0.39816,0.000524615,#DIV/0!,0,0.39816,...,#DIV/0!,0,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0,1
MRV,0.421565,0.36419,0.224177,0.133973,1.8805,0.597621,0.317799,0.8639,0.167733,0.429888,...,3.43689,0.0286998,0.0780171,0.0630912,1.62456,0.455963,1.44531,0.718978,0.367866,0
HOTEIS OTHON,0.0241213,0.0220678,0.0705841,0.00714738,0.341738,0.101261,0.296311,0.914868,0.00505243,0.0962081,...,0.236722,0.183042,0.565149,0.0825203,0.312645,0.0155995,-0.240094,0.202186,0.323883,1
JOÃO FORTES,0.0330869,0.0762805,0.0190804,0.0140674,1.73408,0.737272,0.425165,2.30546,0.333535,0.403736,...,-24.8306,-0.0552628,-0.299662,0.0646431,3.99785,1.80859,3.56518,1.84889,0.184417,1
